In [1]:
import pandas as pd
import polars as pl

import numpy as np


In [2]:
COLUMNAS_POLARS = {
    "COD_HOSPITAL": pl.Int32,
    "CIP_ENCRIPTADO": pl.Int32,
    "SEXO": pl.Categorical,
    "FECHA_NACIMIENTO": pl.Date,
    "ETNIA": pl.Categorical,
    "PROVINCIA": pl.Categorical,
    "COMUNA": pl.Categorical,
    "NACIONALIDAD": pl.Categorical,
    "PREVISION": pl.Categorical,
    "SERVICIO_SALUD": pl.Categorical,
    "TIPO_PROCEDENCIA": pl.Categorical,
    "TIPO_INGRESO": pl.Categorical,
    "ESPECIALIDAD_MEDICA": pl.Categorical,
    "TIPO_ACTIVIDAD": pl.Categorical,
    "FECHA_INGRESO": pl.Date,
    "SERVICIOINGRESO": pl.Categorical,
    "FECHATRASLADO1": pl.Date,
    "SERVICIOTRASLADO1": pl.Categorical,
    "FECHATRASLADO2": pl.Date,
    "SERVICIOTRASLADO2": pl.Categorical,
    "FECHATRASLADO3": pl.Date,
    "SERVICIOTRASLADO3": pl.Categorical,
    "FECHATRASLADO4": pl.Date,
    "SERVICIOTRASLADO4": pl.Categorical,
    "FECHATRASLADO5": pl.Date,
    "SERVICIOTRASLADO5": pl.Categorical,
    "FECHATRASLADO6": pl.Date,
    "SERVICIOTRASLADO6": pl.Categorical,
    "FECHATRASLADO7": pl.Date,
    "SERVICIOTRASLADO7": pl.Categorical,
    "FECHATRASLADO8": pl.Date,
    "SERVICIOTRASLADO8": pl.Categorical,
    "FECHATRASLADO9": pl.Date,
    "SERVICIOTRASLADO9": pl.Categorical,
    "FECHAALTA": pl.Date,
    "SERVICIOALTA": pl.Categorical,
    "TIPOALTA": pl.Categorical,
    "CONDICIONDEALTANEONATO1": pl.Categorical,
    "PESORN1": pl.Int16,
    "SEXORN1": pl.Categorical,
    "RN1ESTADO": pl.Int8,
    "CONDICIONDEALTANEONATO2": pl.Categorical,
    "PESORN2": pl.Int16,
    "SEXORN2": pl.Categorical,
    "RN2ESTADO": pl.Int8,
    "CONDICIONDEALTANEONATO3": pl.Categorical,
    "PESORN3": pl.Int16,
    "SEXORN3": pl.Categorical,
    "RN3ESTADO": pl.Int8,
    "CONDICIONDEALTANEONATO4": pl.Categorical,
    "PESORN4": pl.Int16,
    "SEXORN4": pl.Categorical,
    "RN4ESTADO": pl.Int8,
    "DIAGNOSTICO1": str,
    "DIAGNOSTICO2": str,
    "DIAGNOSTICO3": str,
    "DIAGNOSTICO4": str,
    "DIAGNOSTICO5": str,
    "DIAGNOSTICO6": str,
    "DIAGNOSTICO7": str,
    "DIAGNOSTICO8": str,
    "DIAGNOSTICO9": str,
    "DIAGNOSTICO10": str,
    "DIAGNOSTICO11": str,
    "DIAGNOSTICO12": str,
    "DIAGNOSTICO13": str,
    "DIAGNOSTICO14": str,
    "DIAGNOSTICO15": str,
    "DIAGNOSTICO16": str,
    "DIAGNOSTICO17": str,
    "DIAGNOSTICO18": str,
    "DIAGNOSTICO19": str,
    "DIAGNOSTICO20": str,
    "DIAGNOSTICO21": str,
    "DIAGNOSTICO22": str,
    "DIAGNOSTICO23": str,
    "DIAGNOSTICO24": str,
    "DIAGNOSTICO25": str,
    "DIAGNOSTICO26": str,
    "DIAGNOSTICO27": str,
    "DIAGNOSTICO28": str,
    "DIAGNOSTICO29": str,
    "DIAGNOSTICO30": str,
    "DIAGNOSTICO31": str,
    "DIAGNOSTICO32": str,
    "DIAGNOSTICO33": str,
    "DIAGNOSTICO34": str,
    "DIAGNOSTICO35": str,
    "PROCEDIMIENTO1": str,
    "PROCEDIMIENTO2": str,
    "PROCEDIMIENTO3": str,
    "PROCEDIMIENTO4": str,
    "PROCEDIMIENTO5": str,
    "PROCEDIMIENTO6": str,
    "PROCEDIMIENTO7": str,
    "PROCEDIMIENTO8": str,
    "PROCEDIMIENTO9": str,
    "PROCEDIMIENTO10": str,
    "PROCEDIMIENTO11": str,
    "PROCEDIMIENTO12": str,
    "PROCEDIMIENTO13": str,
    "PROCEDIMIENTO14": str,
    "PROCEDIMIENTO15": str,
    "PROCEDIMIENTO16": str,
    "PROCEDIMIENTO17": str,
    "PROCEDIMIENTO18": str,
    "PROCEDIMIENTO19": str,
    "PROCEDIMIENTO20": str,
    "PROCEDIMIENTO21": str,
    "PROCEDIMIENTO22": str,
    "PROCEDIMIENTO23": str,
    "PROCEDIMIENTO24": str,
    "PROCEDIMIENTO25": str,
    "PROCEDIMIENTO26": str,
    "PROCEDIMIENTO27": str,
    "PROCEDIMIENTO28": str,
    "PROCEDIMIENTO29": str,
    "PROCEDIMIENTO30": str,
    "MEDICOINTERV1_ENCRIPTADO": pl.Int32,
    "FECHAPROCEDIMIENTO1": pl.Date,
    "FECHAINTERV1": pl.Date,
    "ESPECIALIDADINTERVENCION": pl.Categorical,
    "MEDICOALTA_ENCRIPTADO": pl.Int32,
    "USOSPABELLON": str,
    "IR_29301_COD_GRD": pl.Int32,
    "IR_29301_PESO": str,
    "IR_29301_SEVERIDAD": pl.Int8,
    "IR_29301_MORTALIDAD": pl.Int8,
    "HOSPPROCEDENCIA": str,
}

VALORES_NULOS_COLUMNAS = {
    "CIP_ENCRIPTADO": "SIN INFORMACIÓN",
    "PROCEDIMIENTO1": "DESCONOCIDO",
    "PROCEDIMIENTO2": "DESCONOCIDO",
    "PROCEDIMIENTO3": "DESCONOCIDO",
    "PROCEDIMIENTO5": "DESCONOCIDO",
    "IR_29301_COD_GRD": "DESCONOCIDO",
    "IR_29301_PESO": "DESCONOCIDO",
    "IR_29301_SEVERIDAD": "DESCONOCIDO",
    "IR_29301_MORTALIDAD": "DESCONOCIDO",
}

In [3]:
with pl.StringCache():
    dfp = pl.scan_csv(
        "input/*.txt",
        separator="|",
        dtypes=COLUMNAS_POLARS,
        null_values=VALORES_NULOS_COLUMNAS,
    )

    peso_en_float = pl.col("IR_29301_PESO").str.replace(",", ".").cast(pl.Float32, strict=True)
    estancia = pl.col("FECHAALTA") - pl.col("FECHA_INGRESO")
    anio = pl.col("FECHAALTA").dt.year()

    dfp = dfp.with_columns(
        [
            peso_en_float.alias("IR_29301_PESO"),
            estancia.alias("ESTANCIA"),
            anio.alias("ANIO_EGRESO"),
        ]
    )
    dfp = dfp.collect()


In [9]:
j95_pandas = dfp.filter(
    pl.col('DIAGNOSTICO1') == "J95.5"
).to_pandas()

In [24]:
j95_pandas

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,ESPECIALIDADINTERVENCION,MEDICOALTA_ENCRIPTADO,USOSPABELLON,IR_29301_COD_GRD,IR_29301_PESO,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,HOSPPROCEDENCIA,ESTANCIA,ANIO_EGRESO
0,112103,1441678,HOMBRE,1949-11-21,NINGUNO,OSORNO,OSORNO,CHILE,FONASA INSTITUCIONAL - (MAI) B,OSORNO,...,CIRUGÍA TÓRAX,6837,1,41103,3.5129,3,1,HOSPITAL BASE SAN JOSÉ DE OSORNO,23 days,2019
1,112103,231260,MUJER,1945-03-31,NINGUNO,CACHAPOAL,OLIVAR,CHILE,FONASA INSTITUCIONAL - (MAI) B,LIBERTADOR B. O HIGGINS,...,CIRUGÍA TÓRAX,17054,1,41201,1.0413,1,1,None,5 days,2019
2,112102,113277,MUJER,2015-10-05,NINGUNO,CHILOE,PUQUELDÓN,CHILE,FONASA INSTITUCIONAL - (MAI) A,CHILOÉ,...,CIRUGÍA TÓRAX,2932,1,31203,6.1849,3,3,None,22 days,2019
3,112103,402793,MUJER,1989-11-06,NINGUNO,LLANQUIHUE,PUERTO MONTT,CHILE,FONASA INSTITUCIONAL - (MAI) D,DEL RELONCAVÍ,...,CIRUGÍA TÓRAX,6837,1,41203,2.1123,3,2,None,8 days,2019
4,112103,811474,MUJER,1954-11-15,NINGUNO,SANTIAGO,ESTACIÓN CENTRAL,CHILE,FONASA INSTITUCIONAL - (MAI) B,METROPOLITANO CENTRAL,...,NaN,16837,None,44213,0.9628,3,2,None,10 days,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,112103,77275075,HOMBRE,1953-04-02,NINGUNO,MAGALLANES,PUNTA ARENAS,CHILE,FONASA INSTITUCIONAL - (MAI) C,MAGALLANES,...,CIRUGÍA DE TÓRAX,81628269,1,44213,0.9628,3,3,None,26 days,2021
625,114101,69214616,MUJER,1959-11-28,NINGUNO,CORDILLERA,PUENTE ALTO,CHILE,FONASA INSTITUCIONAL - (MAI) A,METROPOLITANO SURORIENTE,...,OTORRINOLARINGOLOGÍA,76357608,2,42220,0.5433,0,0,None,0 days,2021
626,102100,67830226,HOMBRE,1956-05-16,NINGUNO,IQUIQUE,IQUIQUE,CHILE,FONASA INSTITUCIONAL - (MAI) B,IQUIQUE,...,NaN,87683427,None,44211,0.4203,1,1,None,5 days,2021
627,112102,73137515,HOMBRE,2006-08-27,NINGUNO,LINARES,LINARES,CHILE,FONASA INSTITUCIONAL - (MAI) B,DEL MAULE,...,NaN,98438434,1,31201,1.0164,1,1,HOSPITAL DR. CÉSAR GARAVAGNO BUROTTO (TALCA),2 days,2021


In [23]:
j95_pandas.groupby(['ANIO_EGRESO', 'COD_HOSPITAL'])['PROCEDIMIENTO1'].value_counts().reset_index(name='conteo').to_excel('j95_prueba.xlsx')

In [78]:
df = dfp.to_pandas()

In [88]:
procedimientos = df.loc[:, "PROCEDIMIENTO1":"PROCEDIMIENTO30"]
contiene_nuestros_proc = procedimientos.isin(["31.1", "31.29"]).sum(axis=1).astype(bool)
proc_nacionales = df[contiene_nuestros_proc]


In [89]:
metricas = (
    proc_nacionales.groupby(["ANIO_EGRESO", "SERVICIO_SALUD"])
    .agg(
        n_egresos=("COD_HOSPITAL", "count"),
        peso_medio=("IR_29301_PESO", "mean"),
        estancia_media=("ESTANCIA", "mean"),
    )
    .sort_values(["ANIO_EGRESO", "n_egresos"], ascending=False)
)

In [91]:
metricas.to_excel("resumen_traqueostomia_hist.xlsx")
proc_nacionales.to_excel("crudo_traqueostomias_nacionales_hist.xlsx")